# Table of Contents

- [Imports and Data Preprocessing](#Imports-and-Data-Preprocessing)
- [Congestion Calculation](#Congestion-Calculation)
- [SIR Model Fitment](#SIR-Model-Fitment)

# Imports and Data Preprocessing

In [ ]:
import sqlite3
import copy
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
import contextily as cx

In [ ]:
# GLOBAL VARIABLES/CONSTANTS

# SQLite File Path formatter
__SQLITE_PATH_FORMAT = "<PATH_TO_DATABASE>.sqlite"

# sections.shp File Path
__SECTION_SHP = "<PATH_TO_SECTIONS_FILE>.shp"

# Number of experiments
__NUM_EXP = 11

# number of links: 4433

# Output File Directory
__OUTPUT = "output/"

# SQL Query to be excecuted for different tables
__SQL_EXTRACT_MISECT_QUERY = 'SELECT * FROM MISECT'
__SQL_EXTRACT_MILANE_QUERY = 'SELECT * FROM MILANE'

# Columns to extract from different tables
__MISECT_COLUMNS = ['ent', 'eid', 'sid', 'flow_capacity', 'speed', 'travel', 'traveltime', 'density', 'flow', 'dtime']
__MILANE_COLUMNS = ['ent', 'eid', 'sid', 'lane', 'flow', 'speed', 'density', 'input_flow']

# Actual time for each time step
__TIME_REAL = ['14:15', '14:30', '14:45', '15:00', '15:15', '15:30', '15:45', '16:00', '16:15', '16:30', '16:45', '17:00', '17:15', '17:30', '17:45', '18:00', '18:15', '18:30', '18:45', '19:00', '19:15', '19:30', '19:45', '20:00']


# Scenario Names
__SCENARIOS = [
    "0% Dynamic En-Route",
    "10% Dynamic En-Route",
    "20% Dynamic En-Route",
    "30% Dynamic En-Route",
    "40% Dynamic En-Route",
    "50% Dynamic En-Route",
    "60% Dynamic En-Route",
    "70% Dynamic En-Route",
    "80% Dynamic En-Route",
    "90% Dynamic En-Route",
    "100% Dynamic En-Route"
]
__SCENARIO_PRECENTAGES = [
    "0%",
    "10%",
    "20%",
    "30%",
    "40%",
    "50%",
    "60%",
    "70%",
    "80%",
    "90%",
    "100%",
]

In [ ]:
# Create a SQL connection to our SQLite database

# A list of established connections to our databases
con = []

for i in range(__NUM_EXP):
    con.append(sqlite3.connect(__SQLITE_PATH_FORMAT.format(number=i)))

In [ ]:
# Run SQL query and convert SQL to DataFrame

# List of dataframes extracted from each experiment
df = []
# df_milane = []
for i in range(__NUM_EXP):
    # Run SQL
    query = pd.read_sql(__SQL_EXTRACT_MISECT_QUERY, con[i])
    
    # Convert SQL to DataFrame
    dataframe = pd.DataFrame(query, columns = __MISECT_COLUMNS)
    df.append(dataframe)
    
    # query = pd.read_sql(__SQL_EXTRACT_MILANE_QUERY, con[i])
    # dataframe = pd.DataFrame(query, columns = __MILANE_COLUMNS)
    # df_milane.append(dataframe)

In [ ]:
# Read the sections.shp shapefile
sections = gpd.read_file(__SECTION_SHP)
sections.crs

In [ ]:
# Create a deep copy of df as back up in order not to rerun the above cell
df_copy = copy.deepcopy(df)
# df_milane_copy = copy.deepcopy(df_milane)
sections_copy = copy.deepcopy(sections)

In [ ]:
# Restore the sections file in case of modification
sections = copy.deepcopy(sections_copy)
sections = sections.rename(columns={'speed': 'speed_limit'})
df = copy.deepcopy(df_copy)

In [ ]:
# Drop the sections with missing average speed
df_total = []
df_local = []
df_throu = []
print([df[i].shape for i in range(__NUM_EXP)])
for i in range(__NUM_EXP):
    df[i] = df[i][df[i]['speed'] >= 0.0]
    df_total.append(copy.deepcopy(df[i][df[i]['sid'] == 0]))
    df_local.append(copy.deepcopy(df[i][df[i]['sid'] == 1]))
    df_throu.append(copy.deepcopy(df[i][df[i]['sid'] == 2]))
print([df[i].shape for i in range(__NUM_EXP)])
print([df_total[i].shape for i in range(__NUM_EXP)])
print([df_local[i].shape for i in range(__NUM_EXP)])
print([df_throu[i].shape for i in range(__NUM_EXP)])

# Congestion Calculation
- [Table of Contents](#Table-of-Contents)

In [ ]:
def plot_experiment(gdf_list, col_name):
    """
    Plot all the curves of the data for a list of GeoDataFrames on the designated column.
    
    Keyword arguments:
    gdf_list --- a list of GeoDataFrame that contains the processed data 
                 merged from sections.shp and sqlite output from aimsun
    col_name --- the name of the column to be plotted
    """ 
    for i in range(len(gdf_list)):
        plt.plot(np.array(gdf_list[i][col_name]))

In [ ]:
# Preprocess dataframe to merge with sections

group_cols = ['ent','eid']
# identify the columns which we want to average; this could
# equivalently be defined as list(df.columns[4:])
metric_cols = ['flow_capacity']

# create a new DataFrame with a MultiIndex consisting of the group_cols
# and a column for the mean of each column in metric_cols
aggs = []
for i in range(__NUM_EXP):
    aggs.append(df_total[i].groupby(group_cols)[metric_cols].mean())

# 1. remove the metric_cols from df because we are going to replace them
# with the means in aggs 
# 2. dedupe to leave only one row with each combination of group_cols
# in df
for i in range(__NUM_EXP):
    # Step 1
    df_total[i].drop(metric_cols, axis=1, inplace=True)
    
    # Step 2
    # df[i].drop_duplicates(subset=group_cols, keep='last', inplace=True)

# add the mean columns from aggs into df
for i in range(__NUM_EXP):
    df_total[i] = df_total[i].merge(right=aggs[i], right_index=True, left_on=group_cols, how='right')

In [ ]:
# Merge datasets: sections and dataframe
sections_cong = []

for i in range(__NUM_EXP):
    sections_cong.append(pd.merge(df_total[i], sections, how='left', left_on='eid', right_on='eid'))
    
for i in range(__NUM_EXP):
    sections_cong[i] = sections_cong[i][['ent', 'eid', 'speed', 'speed_limit', 'geometry']]

In [ ]:
# Convert the merged sections into GeoDataFrame and drop null values
gdf = []

for i in range(__NUM_EXP):
    gdf.append(gpd.GeoDataFrame(sections_cong[i], geometry='geometry'))
    gdf[i]['speed'] = gdf[i]['speed'].dropna()
    gdf[i]['length'] = gdf[i]['geometry'].length

In [ ]:
# Threshold for a section to be considered congested
rho = np.linspace(0.1, 0.9, 9)
rho

In [ ]:
# Add a column for each section at each threshold,
# congested = 1, else 0
for threshold in rho:
    for i in range(__NUM_EXP):
        speed_ratio = gdf[i]['speed'] / gdf[i]['speed_limit']
        gdf[i]['congested at rho = ' + str(round(threshold, 1))] = [int(r < threshold) for r in speed_ratio]

In [ ]:
# Add a column for congestion weight at each threshold,
# congested = 1, else 0
for threshold in rho:
    for i in range(__NUM_EXP):
        weight = gdf[i]['length'] * gdf[i]['congested at rho = ' + str(round(threshold, 1))]
        gdf[i]['weight at rho = ' + str(round(threshold, 1))] = weight

In [ ]:
# Group each GeoDataFrame on timestep and aggregate by sum
# Remove the first row as it is the average of the rest
gdf_agg = []

for i in range(__NUM_EXP):
    gdf_agg.append(gdf[i].groupby('ent').agg(np.sum).iloc[1:, :])

In [ ]:
for threshold in rho:
    for i in range(__NUM_EXP):
        gdf_agg[i]['congestion ratio at rho = ' + str(round(threshold, 1))] = gdf_agg[i]['weight at rho = ' + str(round(threshold, 1))] / total_length

# SIR Model Fitment

In [ ]:
# This model is adapted from the following source:
# Marisa Eisenberg (marisae@umich.edu)
# Yu-Han Kao (kaoyh@umich.edu) -7-9-17
# SIR model example for python 2.7, with the following author updated to python 3.10
# Qianxin (Carl) Gan (<hidden>@berkeley.edu)

#### Import all the packages ####
import scipy.optimize as optimize
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import copy

from scipy.integrate import odeint as ode
from scipy.stats import poisson
from scipy.stats import norm

## Estimating $R_0$ using Congested Link Counts

In [ ]:
# model equations for the scaled SIR model

def model(ini, time_step, params):
    Y = np.zeros(3) #column vector for the state variables
    X = ini
    mu = 0
    beta = params[0]
    gamma = params[1]

    Y[0] = mu - beta * X[0] * X[1] - mu * X[0] #S
    Y[1] = beta * X[0] * X[1] - gamma * X[1] - mu * X[1] #I
    Y[2] = gamma * X[1] - mu * X[2] #R

    return Y

def x0fcn(params, data):
    S0 = 1.0 - (data[0] / params[2])
    I0 = data[0] / params[2]
    R0 = 0.0
    X0 = [S0, I0, R0]

    return X0


def yfcn(res, params):
    return res[:,1]*params[2]


# Simplified FIM (Fisher information matirx) function for the SIR model

def minifisher (times, params, data, delta = 0.001):
    #params = np.array(params)
    listX = []
    params_1 = np.array(params)
    params_2 = np.array(params)
    for i in range(len(params)):
        params_1[i] = params[i] * (1 + delta)
        params_2[i]= params[i] * (1 - delta)

        res_1 = ode(model, x0fcn(params_1, data), times, args=(params_1,))
        res_2 = ode(model, x0fcn(params_2, data), times, args=(params_2,))
        subX = (yfcn(res_1, params_1) - yfcn(res_2, params_2)) / (2 * delta * params[i])
        listX.append(subX.tolist())
    X = np.matrix(listX)
    FIM = np.dot(X, X.transpose())
    return FIM



# cost function for the SIR model

def NLL(params, data, times): #negative log likelihood
    params = np.abs(params)
    data = np.array(data)
    res = ode(model, x0fcn(params, data), times, args=(params,))
    y = yfcn(res, params)
    # nll = sum(y) - sum(data * np.log(y))
    # note this is a slightly shortened version--there's an additive constant term missing but it 
    # makes calculation faster and won't alter the threshold. Alternatively, can do:
    nll = -sum(np.log(poisson.pmf(np.round(data), np.round(y)))) # the round is b/c Poisson is for (integer) count data
    # this can also barf if data and y are too far apart because the dpois will be ~0, which makes the log angry

    # ML using normally distributed measurement error (least squares)
    # nll = -sum(np.log(norm.pdf(data,y,0.1*np.mean(data)))) # example WLS assuming sigma = 0.1*mean(data)
    # nll = sum((y - data)**2)  # alternatively can do OLS but note this will mess with the thresholds 
    #                             for the profile! This version of OLS is off by a scaling factor from
    #                             actual LL units.
    return nll



# Profile Likelihood Generator

# Input definitions
# params = starting parameters (all, including the one to be profiled)
# profparam = index within params for the parameter to be profiled
#   ---reminder to make this allow you to pass the name instead later on
# costfun = cost function for the model - should include params, times, and data as arguments.
#   Note costfun doesn't need to be specially set up for fixing the profiled parameter, 
#   it's just the regular function you would use to estimate all the parameters
#   (it will get reworked to fix one of them inside ProfLike)
# times, data = data set (times & values, or whatever makes sense)
#   ---possibly change this so it's included in costfun and not a separate set of inputs? Hmm.
# perrange = the percent/fraction range to profile the parameter over (default is 0.5)
# numpoints = number of points to profile at in each direction (default is 10)

# Output
# A list with:
#   - profparvals: the values of the profiled parameter that were used
#   - fnvals: the cost function value at each profiled parameter value
#   - convergence: the convergence value at each profiled parameter value
#   - paramestvals: the estimates of the other parameters at each profiled parameter value

def proflike(params, profindex, cost_func, times, data, perrange = 0.5, numpoints = 10):
    profrangedown = np.linspace(params[profindex], params[profindex] * (1 - perrange), numpoints).tolist()
    profrangeup = np.linspace(params[profindex], params[profindex] * (1 + perrange), numpoints).tolist()[1:] #skip the duplicated values
    profrange = [profrangedown, profrangeup]
    currvals = []
    currparams = []
    currflags = []

    def profcost(fit_params, profparam, profindex, data, times, cost_func):
        paramstest = fit_params.tolist()
        paramstest.insert(profindex, profparam)
        return cost_func(paramstest, data, times)

    fit_params = params.tolist() #make a copy of params so we won't change the origianl list
    fit_params.pop(profindex)
    # print('Starting profile...')
    for i in range(len(profrange)):
        for j in profrange[i]:
            # print(i, j)
            optimizer = optimize.minimize(profcost, fit_params, args=(j, profindex, data, times, cost_func), method='Nelder-Mead')
            fit_params = np.abs(optimizer.x).tolist() #save current fitted params as starting values for next round
            #print optimizer.fun
            currvals.append(optimizer.fun)
            currflags.append(optimizer.success)
            currparams.append(np.abs(optimizer.x).tolist())

    # structure the return output
    profrangedown.reverse()
    out_profparam = profrangedown + profrangeup
    temp_ind = list(range(len(profrangedown)))
    temp_ind.reverse()
    out_params = [currparams[i] for i in temp_ind] + currparams[len(profrangedown):]
    out_fvals = [currvals[i] for i in temp_ind] + currvals[len(profrangedown):]
    out_flags = [currflags[i] for i in temp_ind] + currflags[len(profrangedown):]
    output = {'profparam': out_profparam, 'fitparam': np.array(out_params), 'fcnvals': out_fvals, 'convergence': out_flags}
    return output

In [ ]:
def estimate_params(data, beta=0.4, gamma=0.25, N=4433):
    """
    Estimate the parameters for the SIR model
    
    Parameters
    ----------
    data: list or np.array
        congested link count at each timestep
    times: np.array
        array of time steps
        
    Returns
    -------
    result_estimations: dict
        a dictionary of parameter name and estimated value pairs
    """
    #### Set initial parameter values and initial states ####
    times=np.linspace(0, len(data) - 1, len(data))
    params = [beta, gamma, N]  # make sure all the params and inition states are float
    paramnames = ['beta', 'gamma', 'k']
    # ini = x0fcn(params, data)
    
    #### Simulate the model ####
    # res = ode(model, ini, times, args=(params,))
    
    #### Parameter estimation ####
    optimizer = optimize.minimize(NLL, params, args=(data, times), method='Nelder-Mead')
    paramests = np.abs(optimizer.x)
    
    #### Calculate the simplified Fisher Information Matrix (FIM) ####
    # FIM = minifisher(times, params, data, delta = 0.001)
    
    #### Generate profile likelihoods and confidence bounds ####
    # threshold = stats.chi2.ppf(0.95,len(paramests)) / 2.0 + optimizer.fun
    perrange = 0.25 #percent range for profile to run across

    profiles = {}
    result_estimations = {}
    for i in range(len(paramests)):
        profiles[paramnames[i]] = proflike(paramests, i, NLL, times, data, perrange=perrange)
        paramnames_fit = [n for n in paramnames if n not in [paramnames[i]]]
        paramests_fit = [v for v in paramests if v not in [paramests[i]]]
        
        assert len(paramnames_fit) == len(paramests_fit)
        for i in range(len(paramnames_fit)):
            result_estimations[paramnames_fit[i]] = paramests_fit[i]
            
    return result_estimations

In [ ]:
# Calculate R_0 for all scenarios
# where each row corresponds to each experiment
# and column corresponds to rho values
beta_matrix = []
gamma_matrix = []
R_0_matrix = []

for i in range(__NUM_EXP):
    beta_array = []
    gamma_array = []
    R_0_array = []
    
    for threshold in rho:
        congestion_count = gdf_agg[i]['congested at rho = ' + str(round(threshold, 1))].tolist()
        estimated_parameters = estimate_params(congestion_count, N=4433)
        
        beta = estimated_parameters['beta']
        gamma = estimated_parameters['gamma']
        R_0 = beta / gamma
        
        beta_array.append(beta)
        gamma_array.append(gamma)
        R_0_array.append(R_0)
    beta_matrix.append(beta_array)
    gamma_matrix.append(gamma_array)
    R_0_matrix.append(R_0_array)

In [ ]:
# plot results
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111)
# ax.set_prop_cycle('color', [cm(1.*i/NUM_COLORS) for i in range(NUM_COLORS)])
ax.set_prop_cycle('color', plt.cm.tab20(np.linspace(0, 1, __NUM_EXP)))
x = np.arange(0.1, 1.0, 0.1)
for i in range(__NUM_EXP):
    plt.plot(x, R_0_matrix[i])
plt.legend(
    __SCENARIO_PRECENTAGES,
    title='Dynamic En-Route Percentage',
)
plt.xlabel(r'$\rho$', fontsize=18)
plt.ylabel(r'$R_0$', fontsize=18)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.title(r'$R_0$ for All Experiments', fontsize=20)
plt.savefig(__OUTPUT + 'R0_for_all_experiments_N_4433.png')
plt.show()

In [ ]:
# plot results
rho_num = 9
plt.figure(figsize=(10, 6))
x = np.arange(__NUM_EXP)
y = np.array(R_0_matrix)[:, rho_num - 1]

model3 = np.poly1d(np.polyfit(x, y, 3))
polyline = np.linspace(0, 10, 50)
plt.plot(polyline, model3(polyline), color='red', linewidth=3)

plt.plot(x, y, 'k-o', color='blue')

plt.ylabel(r'$R_0$', fontsize='20')
default_x_ticks = range(len(__SCENARIO_PRECENTAGES))
plt.xticks(default_x_ticks, __SCENARIO_PRECENTAGES, fontsize='16')
plt.yticks(fontsize='16')
plt.xlabel('Dynamic En-Route', fontsize='16')
plt.title(fr'$R_0$ for All Scenarios when $\rho$ = 0.{rho_num}', fontsize='24')
plt.legend(['Polynomial Fit', 'Data Points'], fontsize='15')
plt.savefig(__OUTPUT + f'R0_for_all_experiments_rho_{rho_num}_N_4433.png')
plt.show()

In [ ]:
#fit polynomial models up to degree 5
model1 = np.poly1d(np.polyfit(x, y, 1))
model2 = np.poly1d(np.polyfit(x, y, 2))
model3 = np.poly1d(np.polyfit(x, y, 3))
model4 = np.poly1d(np.polyfit(x, y, 4))
# model5 = np.poly1d(np.polyfit(x, y, 5))

#create scatterplot
polyline = np.linspace(0, 10, 50)
plt.scatter(x, y)

#add fitted polynomial lines to scatterplot 
plt.plot(polyline, model1(polyline), color='green')
plt.plot(polyline, model2(polyline), color='red')
plt.plot(polyline, model3(polyline), color='purple')
plt.plot(polyline, model4(polyline), color='blue')
# plt.plot(polyline, model5(polyline), color='orange')
plt.show()

In [ ]:
#define function to calculate adjusted r-squared
def adjR(x, y, degree):
    results = {}
    coeffs = np.polyfit(x, y, degree)
    p = np.poly1d(coeffs)
    yhat = p(x)
    ybar = np.sum(y)/len(y)
    ssreg = np.sum((yhat-ybar)**2)
    sstot = np.sum((y - ybar)**2)
    results['r_squared'] = 1- (((1-(ssreg/sstot))*(len(y)-1))/(len(y)-degree-1))

    return results

#calculated adjusted R-squared of each model
print(adjR(x, y, 1))
print(adjR(x, y, 2))
print(adjR(x, y, 3))
print(adjR(x, y, 4))
# adjR(x, y, 5)

In [ ]:
beta_matrix_df = pd.DataFrame(beta_matrix)

In [ ]:
gamma_matrix_df = pd.DataFrame(gamma_matrix)

In [ ]:
R_0_matrix_df = pd.DataFrame(R_0_matrix)

# Plotting Fitted SIR Model

In [ ]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

In [ ]:
def plot_fremont_SIR(I0=1, beta=0.25, gamma=0.05, N=4433):
    # Total population, N.
    # Initial number of infected and recovered individuals, I0 and R0.
    # rho = 0.9
    R0 = 0
    # Everyone else, S0, is susceptible to infection initially.
    S0 = N - I0 - R0
    # A grid of time points (in quarters of an hour)
    t = list(range(24))

    # The SIR model differential equations.
    def deriv(y, t, N, beta, gamma):
        S, I, R = y
        dSdt = -beta * S * I / N
        dIdt = beta * S * I / N - gamma * I
        dRdt = gamma * I
        return dSdt, dIdt, dRdt

    # Initial conditions vector
    y0 = S0, I0, R0
    # Integrate the SIR equations over the time grid, t.
    ret = odeint(deriv, y0, t, args=(N, beta, gamma))
    S, I, R = ret.T

    # Plot the data on three separate curves for S(t), I(t) and R(t)
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.plot(t, np.log(S / N), 'b', alpha=0.5, lw=2, label='S(t)')
    ax.plot(t, np.log(I / N), 'r', alpha=0.5, lw=2, label='I(t)')
    ax.plot(t, np.log(R / N), 'g', alpha=0.5, lw=2, label='R(t)')
    ax.set_ylabel('Log Percentage', fontsize=16)
    # ax.set_xlabel('Time', fontsize=16)
    # ax.set_yticklabels(ax.get_yticklabels(), fontsize=16)
    ax.yaxis.set_tick_params(length=0)
    ax.xaxis.set_tick_params(length=0)
    ax.tick_params(axis='x', labelsize=16)
    ax.tick_params(axis='y', labelsize=16)
    ax.set_xlim(0, 23)
    # ax.set_yscale("log")
    
    ax.yaxis.set_ticks(np.arange(-4, 0.5, 1.0))
    ax.set_xticklabels(np.array(__TIME_REAL)[::5], rotation=90, fontsize=16)
    ax.set_yticklabels(['1e-4', '1e-3', '1e-2', '1e-1', '1'], fontsize=16)
    print(ax.get_xticks())
    
    ax.grid(visible=True, which='major', c='w', lw=2, ls='-')
    legend = ax.legend(fontsize=12)
    legend.get_frame().set_alpha(0.5)
    # ax.grid(visible=True, which='major', c='w', lw=2, ls='-')
    plt.savefig(__OUTPUT + f'SIR_model_beta_{beta}_gamma_{gamma}_{src_num}_{rho_num}.png')
    plt.show()

In [ ]:
src_num = 9
rho_num = 9
plot_fremont_SIR(gdf_agg[src_num][f'congested at rho = 0.{rho_num}'].tolist()[0], beta=beta_matrix[src_num][rho_num - 1], gamma=gamma_matrix[src_num][rho_num - 1])

In [ ]:
# plot beta and gamm estimated results with fixed rho
plt.figure(figsize=(10, 6))
x = np.arange(__NUM_EXP)
plt.plot(x, np.array(gamma_matrix)[:, 8])
# plt.xticks([
#     "0% SRC(All SUE)",
#     "10% SRC",
#     "20% SRC",
#     "30% SRC",
#     "40% SRC",
#     "50% SRC",
#     "60% SRC",
#     "70% SRC",
#     "80% SRC",
#     "90% SRC",
#     "100% All SRC (No SUE)"
# ])
plt.xlabel(r'$\rho$')
plt.ylabel(r'$\beta$')
plt.title(r'$\beta$ for All $\rho$')
# plt.savefig(__OUTPUT + 'beta_for_all_experiments.png')
plt.show()

In [ ]:
# plot results
plt.figure(figsize=(10, 6))
x = np.arange(0.1, 1.0, 0.1)

for i in range(__NUM_EXP):
    plt.plot(x, beta_matrix[i])
plt.legend([
    "0% SRC(All SUE)",
    "10% SRC",
    "20% SRC",
    "30% SRC",
    "40% SRC",
    "50% SRC",
    "60% SRC",
    "70% SRC",
    "80% SRC",
    "90% SRC",
    "100% All SRC (No SUE)"
])
plt.xlabel(r'$\rho$')
plt.ylabel(r'$\beta$')
plt.title(r'$\beta$ for All $\rho$')
# plt.savefig(__OUTPUT + 'beta_for_all_experiments.png')
plt.show()

In [ ]:
# plot results
plt.figure(figsize=(10, 6))
x = np.arange(0.1, 1.0, 0.1)
plt.plot(x, np.mean(beta_matrix, axis=0))
plt.plot(x, np.mean(gamma_matrix, axis=0))
# for i in range(__NUM_EXP):
#     plt.plot(x, np.mean(beta_matrix[i]))
#     plt.plot(x, gamma_matrix[i])
plt.legend([
    r'$\beta$',
    r'$\gamma$'
])
plt.xlabel(r'$\rho$')
plt.ylabel(r'$\gamma$')
plt.title(r'Estimated Parameters for All $\rho$')
plt.savefig(__OUTPUT + 'Estimated Parameters for All $\rho$.png')
plt.show()